In [2]:
from datasets import load_dataset, Dataset, load_from_disk
import pandas as pd
import numpy as np

/home/anders/codetrace/mechinterp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Seperate out all of the java from everything else

In [3]:
dataset = load_dataset("nuprl/MultiPL-E-synthetic-solutions") 
df = pd.DataFrame(dataset["train"].filter(lambda example: example["language"] == "java"))
df.head()

Found cached dataset parquet (/home/anders/.cache/huggingface/datasets/nuprl___parquet/nuprl--MultiPL-E-synthetic-solutions-d2ef7ed8282810ca/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 1/1 [00:00<00:00, 657.11it/s]
Loading cached processed dataset at /home/anders/.cache/huggingface/datasets/nuprl___parquet/nuprl--MultiPL-E-synthetic-solutions-d2ef7ed8282810ca/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-089f817c1c829bd9.arrow


,name,language,prompt,solution
0,HumanEval_0_has_close_elements,java,import java.util.*;\nimport java.lang.reflect....,// Sort the list in ascending order\n ...
1,HumanEval_100_make_a_pile,java,import java.util.*;\nimport java.lang.reflect....,ArrayList<Long> levels = new ArrayList...
2,HumanEval_101_words_string,java,import java.util.*;\nimport java.lang.reflect....,ArrayList<String> al = new ArrayList<S...
3,HumanEval_102_choose_num,java,import java.util.*;\nimport java.lang.reflect....,long chosen = -1;\n for (long i...
4,HumanEval_104_unique_digits,java,import java.util.*;\nimport java.lang.reflect....,ArrayList<Long> b = new ArrayList<Long...


In [4]:
df["code"] = df["prompt"] + df["solution"]
df = df.drop(columns=[ "language", "prompt", "solution"])
df.head()

,name,code
0,HumanEval_0_has_close_elements,import java.util.*;\nimport java.lang.reflect....
1,HumanEval_100_make_a_pile,import java.util.*;\nimport java.lang.reflect....
2,HumanEval_101_words_string,import java.util.*;\nimport java.lang.reflect....
3,HumanEval_102_choose_num,import java.util.*;\nimport java.lang.reflect....
4,HumanEval_104_unique_digits,import java.util.*;\nimport java.lang.reflect....


Cut off all examples at first instance of an int

In [5]:
delim = ["int "] #remember to add space after each delimiter

In [6]:
condition = df["code"].str.contains("int ")
print(np.any(condition))

True


In [7]:
df_condition = df[condition]
df_condition.head()

,name,code
0,HumanEval_0_has_close_elements,import java.util.*;\nimport java.lang.reflect....
4,HumanEval_104_unique_digits,import java.util.*;\nimport java.lang.reflect....
5,HumanEval_105_by_length,import java.util.*;\nimport java.lang.reflect....
7,HumanEval_108_count_nums,import java.util.*;\nimport java.lang.reflect....
8,HumanEval_109_move_one_ball,import java.util.*;\nimport java.lang.reflect....


Takes the code, and checks for the pattern "int [any number of things] = ", and then puts everything that comes before the pattern into the "prompt" category, and the after into the "solution" category. 

In [30]:
df_condition[['prompt', 'delim', 'solution']] = df_condition["code"].str.split(pat = r'(int\s\S*\s=\s)', n= 1, expand=True, regex=True)
df_condition.head()

,name,code,prompt,solution,completion,delim
0,HumanEval_0_has_close_elements,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,0; i < numbers.size() - 1; i++) {\n ...,0,int i =
4,HumanEval_104_unique_digits,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,0; i < x.size(); i++) {\n if (x.get...,0,int i =
7,HumanEval_108_count_nums,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,"0;\n if (str.startsWith(""-"")) {\n ...",0,int sum =
8,HumanEval_109_move_one_ball,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,0; i < arr.size(); i++) {\n ArrayLi...,0,int i =
9,HumanEval_10_make_palindrome,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,0; index < string.length(); index++) {\n ...,0,int index =


In [31]:
df_condition["prompt"] = df_condition["prompt"] + df_condition["delim"]

In [32]:
df_condition.head()

,name,code,prompt,solution,completion,delim
0,HumanEval_0_has_close_elements,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,0; i < numbers.size() - 1; i++) {\n ...,0,int i =
4,HumanEval_104_unique_digits,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,0; i < x.size(); i++) {\n if (x.get...,0,int i =
7,HumanEval_108_count_nums,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,"0;\n if (str.startsWith(""-"")) {\n ...",0,int sum =
8,HumanEval_109_move_one_ball,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,0; i < arr.size(); i++) {\n ArrayLi...,0,int i =
9,HumanEval_10_make_palindrome,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,0; index < string.length(); index++) {\n ...,0,int index =


Seperates out the value that comes directly after the "int" decleration from the rest of the solution. This is put directly into the "completion" column.

In [9]:
df_condition["completion"] = df_condition['solution'].str.split(';', n= 1, expand=True)[0]
df_condition.head()

/tmp/ipykernel_521455/891727865.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_condition["completion"] = df_condition['solution'].str.split(';', n= 1, expand=True)[0]


,name,code,prompt,solution,completion
0,HumanEval_0_has_close_elements,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,0; i < numbers.size() - 1; i++) {\n ...,0
4,HumanEval_104_unique_digits,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,0; i < x.size(); i++) {\n if (x.get...,0
5,HumanEval_105_by_length,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,None,None
7,HumanEval_108_count_nums,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,"0;\n if (str.startsWith(""-"")) {\n ...",0
8,HumanEval_109_move_one_ball,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,0; i < arr.size(); i++) {\n ArrayLi...,0


In [10]:
df_condition.head()
df_condition["completion"].unique()
condition = df_condition["completion"].notna()
df_condition = df_condition[condition]
df_condition["completion"].isna().value_counts()

completion
False    68
Name: count, dtype: int64

In [11]:
df_condition["completion"].value_counts() #use to get an int that isn't zero

completion
0                                                                             52
1                                                                              2
Integer.parseInt(parts[0])                                                     1
txt.length()                                                                   1
arr.size() - 1                                                                 1
lst.size()                                                                     1
Integer.parseInt(nums[0])                                                      1
planets.indexOf(planet1)                                                       1
Integer.MIN_VALUE                                                              1
b.length()                                                                     1
(int) number                                                                   1
wordsToUniqueCharacters.get(o2).compareTo(wordsToUniqueCharacters.get(o1))     1
l.size()         

In [33]:
df_condition["prompt"].str.contains("for").value_counts() #use for a no-for dataset

new_df = df_condition[~df_condition["prompt"].str.contains("for")]
new_df.head()

,name,code,prompt,solution,completion,delim
18,HumanEval_119_match_parens,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,0;\n int c2 = 0;\n for (int i = ...,0,int c1 =
25,HumanEval_126_is_sorted,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,0;\n int previous_number = 0;\n ...,0,int running_max =
28,HumanEval_134_check_if_last_char_is_a_letter,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,txt.length();\n if (txt.substring(len -...,txt.length(),int len =
33,HumanEval_140_fix_spaces,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,0;\n while (i < result.length() - 2) {\...,0,int i =
35,HumanEval_142_sum_squares,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,lst.size();\n for (int i = 0; i < sz; i...,lst.size(),int sz =


In [16]:
new_df.reset_index()

,index,name,code,prompt,solution,completion
0,18,HumanEval_119_match_parens,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,0;\n int c2 = 0;\n for (int i = ...,0
1,25,HumanEval_126_is_sorted,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,0;\n int previous_number = 0;\n ...,0
2,28,HumanEval_134_check_if_last_char_is_a_letter,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,txt.length();\n if (txt.substring(len -...,txt.length()
3,33,HumanEval_140_fix_spaces,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,0;\n while (i < result.length() - 2) {\...,0
4,35,HumanEval_142_sum_squares,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,lst.size();\n for (int i = 0; i < sz; i...,lst.size()
5,39,HumanEval_147_get_max_triples,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,0;\n ArrayList<Integer> arrayList = new...,0
6,40,HumanEval_148_bf,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,planets.indexOf(planet1);\n int indexPl...,planets.indexOf(planet1)
7,47,HumanEval_154_cycpattern_check,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,b.length();\n boolean found = false;\n ...,b.length()
8,48,HumanEval_156_int_to_mini_roman,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,"(int) number;\n String res = """";\n ...",(int) number
9,57,HumanEval_18_how_many_times,import java.util.*;\nimport java.lang.reflect....,import java.util.*;\nimport java.lang.reflect....,0;\n while (index != -1) {\n ...,0


In [34]:
new_df.drop([ "name", "code", "solution", "delim"], axis=1, inplace=True)
new_df.head()

/tmp/ipykernel_521455/2542724173.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop([ "name", "code", "solution", "delim"], axis=1, inplace=True)


,prompt,completion
18,import java.util.*;\nimport java.lang.reflect....,0
25,import java.util.*;\nimport java.lang.reflect....,0
28,import java.util.*;\nimport java.lang.reflect....,txt.length()
33,import java.util.*;\nimport java.lang.reflect....,0
35,import java.util.*;\nimport java.lang.reflect....,lst.size()


In [35]:
new_df.to_csv('int_dataset_java_no_for.csv',index=False)